In [1]:
import torch

import mowl
mowl.init_jvm('10g')
from mowl.datasets import PathDataset, Dataset

from src.utils import *
from src.elmodule import *

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/lib/python3.10/dist-packages/mowl/lib/slf4j-simple-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/lib/python3.10/dist-packages/mowl/lib/slf4j-log4j12-1.7.3.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.SimpleLoggerFactory]


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
print(device)

cpu


In [3]:
#dataset_name = 'family'
#dataset_name = 'pizza'
dataset_name = 'OWL2DL-1'

In [4]:
experiments = get_experimets(dataset_name)

**Models**

**Eval**

**Experiments: Box2EL**

In [5]:
for experiment in experiments: 
    dataset_name = experiment['dataset_name']
    file_name = experiment['file_name']
    format_ = experiment['format_']
    add_noise = experiment['add_noise']                                                                                                               
   
    dataset = PathDataset(ontology_path=f'datasets/bin/{file_name}_train.owl', # we add noise to train
                          testing_path=f'datasets/bin/{dataset_name}_test.owl',
                          validation_path=f'datasets/bin/{dataset_name}_val.owl')
    
    train_ont = preprocess_ontology_el(dataset.ontology)
    test_ont = preprocess_ontology_el(dataset.testing)
    valid_ont = preprocess_ontology_el(dataset.validation)

    dataset = Dataset(train_ont, testing=test_ont, validation=valid_ont)
    
    box2el = ElModel(dataset, 
                     module_name='box2el', 
                     dim=200, 
                     margin=0.1, 
                     batch_size=4096*8, 
                     test_batch_size=32, 
                     epochs=500, 
                     learning_rate=0.001,
                     device=device)
    
    box2el._train()
    
    print(f'{file_name}:')
    print('Membership:')
    metrics_membership = box2el._eval('membership')
    print('Subsumption:')
    metrics_subsumption = box2el._eval('subsumption')
    print('Link Prediction:')
    metrics_link_prediction = box2el._eval('link_prediction')
    print()
    save_results(metrics_subsumption, metrics_membership, metrics_link_prediction, f'models/results/box2el/{file_name}.txt')

In [5]:
dataset = PathDataset(ontology_path=f'datasets/bin/{dataset_name}_train.owl', 
                      testing_path=f'datasets/bin/{dataset_name}_test.owl',
                      validation_path=f'datasets/bin/{dataset_name}_val.owl')

train_ont = preprocess_ontology_el(dataset.ontology)
test_ont = preprocess_ontology_el(dataset.testing)
valid_ont = preprocess_ontology_el(dataset.validation)

dataset = Dataset(train_ont, testing=test_ont, validation=valid_ont)

box2el = ElModel(dataset, 
                 module_name='box2el', 
                 dim=200, 
                 margin=0.1, 
                 batch_size=4096*8, 
                 test_batch_size=32, 
                 epochs=500, 
                 learning_rate=0.001,
                 device=device)

[main] WARN uk.ac.manchester.cs.owl.owlapi.OWLOntologyManagerImpl - Illegal redeclarations of entities: reuse of entity http://www.w3.org/2000/01/rdf-schema#Literal in punning not allowed [Declaration(NamedIndividual(rdfs:Literal)), Declaration(Class(rdfs:Literal))]
[main] WARN uk.ac.manchester.cs.owl.owlapi.OWLOntologyManagerImpl - Illegal redeclarations of entities: reuse of entity http://www.w3.org/1999/02/22-rdf-syntax-ns#XMLLiteral in punning not allowed [Declaration(NamedIndividual(rdf:XMLLiteral)), Declaration(Class(rdf:XMLLiteral)), Declaration(Datatype(rdf:XMLLiteral))]
[main] WARN uk.ac.manchester.cs.owl.owlapi.OWLOntologyManagerImpl - Illegal redeclarations of entities: reuse of entity http://www.w3.org/2000/01/rdf-schema#Literal in punning not allowed [Declaration(NamedIndividual(rdfs:Literal)), Declaration(Class(rdfs:Literal))]
[main] WARN uk.ac.manchester.cs.owl.owlapi.OWLOntologyManagerImpl - Illegal redeclarations of entities: reuse of entity http://www.w3.org/1999/02/2

In [6]:
metrics_link_prediction = box2el._eval('membership')

MRR: 0.013, Hits@1: 0.001, Hits@5: 0.009, Hits@10: 0.024


In [7]:
metrics_link_prediction = box2el._eval('link_prediction')

MRR: 0.002, Hits@1: 0.000, Hits@5: 0.001, Hits@10: 0.002


In [8]:
box2el._train()

INFO:root:Reverse translation. Ignoring axiom: RangeAxiom*(186 127)
INFO:root:'uk.ac.manchester.cs.owl.owlapi.OWLObjectPropertyRa' object has no attribute 'getSubClass'
INFO:root:Reverse translation. Ignoring axiom: FunctionalObjectProperty*(59)
INFO:root:de.tudresden.inf.lat.jcel.owlapi.translator.TranslationException: The translation map is incomplete. Item id was not found: '59'.
INFO:root:Reverse translation. Ignoring axiom: RangeAxiom*(188 129)
INFO:root:'uk.ac.manchester.cs.owl.owlapi.OWLObjectPropertyRa' object has no attribute 'getSubClass'
INFO:root:Reverse translation. Ignoring axiom: RangeAxiom*(189 131)
INFO:root:'uk.ac.manchester.cs.owl.owlapi.OWLObjectPropertyRa' object has no attribute 'getSubClass'
INFO:root:Reverse translation. Ignoring axiom: FunctionalObjectProperty*(155)
INFO:root:'uk.ac.manchester.cs.owl.owlapi.OWLFunctionalObject' object has no attribute 'getSubClass'
INFO:root:Reverse translation. Ignoring axiom: FunctionalObjectProperty*(156)
INFO:root:'uk.ac.ma

Epoch: 0, Training: EL loss: 3.5926, ABox loss (Membership): 0.8364, ABox loss (Link Prediction): 5.7182
Epoch: 25, Training: EL loss: 3.3893, ABox loss (Membership): 0.8211, ABox loss (Link Prediction): 5.4705
Epoch: 50, Training: EL loss: 3.2337, ABox loss (Membership): 0.8288, ABox loss (Link Prediction): 5.1819
Epoch: 75, Training: EL loss: 3.1209, ABox loss (Membership): 0.8456, ABox loss (Link Prediction): 4.8823
Epoch: 100, Training: EL loss: 3.0466, ABox loss (Membership): 0.8554, ABox loss (Link Prediction): 4.5920
Epoch: 125, Training: EL loss: 2.9614, ABox loss (Membership): 0.8489, ABox loss (Link Prediction): 4.3212
Epoch: 150, Training: EL loss: 2.8868, ABox loss (Membership): 0.8233, ABox loss (Link Prediction): 4.0730
Epoch: 175, Training: EL loss: 2.8081, ABox loss (Membership): 0.7750, ABox loss (Link Prediction): 3.8458
Epoch: 200, Training: EL loss: 2.7380, ABox loss (Membership): 0.6996, ABox loss (Link Prediction): 3.6367
Epoch: 225, Training: EL loss: 2.6920, ABo

In [9]:
metrics_link_prediction = box2el._eval('membership')

MRR: 0.027, Hits@1: 0.000, Hits@5: 0.014, Hits@10: 0.048


In [10]:
metrics_link_prediction = box2el._eval('link_prediction')

MRR: 0.002, Hits@1: 0.000, Hits@5: 0.002, Hits@10: 0.002
